<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty100_Upside_Analysis_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance
!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 760 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.4 MB 4.5 MB/s 


In [2]:
import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Stock_Analysis/data/Nifty100Scrips.csv')

mypf = mypf[mypf['InScope'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

#stock_ids

stock_ids = stock_n100

In [3]:
mypf = mypf[mypf.Forecast.notnull()]
stock_ids = mypf[mypf.Forecast.notnull()]['Symbol'].unique()

In [4]:
stock_ids.sort()
stock_ids

array(['ASIANPAINT', 'AXISBANK', 'BAJFINANCE', 'BERGEPAINT', 'BHARTIARTL',
       'BRITANNIA', 'CANBK', 'CIPLA', 'COLPAL', 'DABUR', 'DEEPAKNTR',
       'DIVISLAB', 'DMART', 'DRREDDY', 'EICHERMOT', 'FEDERALBNK',
       'GODREJCP', 'GODREJIND', 'HAVELLS', 'HCLTECH', 'HDFC', 'HDFCBANK',
       'HDFCLIFE', 'HEROMOTOCO', 'HINDUNILVR', 'IBREALEST', 'ICICIBANK',
       'ICICIPRULI', 'INDIGO', 'INDUSINDBK', 'INFY', 'IRCTC', 'ITC',
       'JUBLFOOD', 'KOTAKBANK', 'LT', 'LTI', 'M&M', 'MARUTI',
       'MCDOWELL-N', 'MINDTREE', 'MOTILALOFS', 'MUTHOOTFIN', 'NATIONALUM',
       'NAUKRI', 'PIDILITIND', 'RELIANCE', 'SAIL', 'SBILIFE', 'SBIN',
       'SUNPHARMA', 'TATACONSUM', 'TATAMOTORS', 'TATAPOWER', 'TATASTEEL',
       'TCS', 'TECHM', 'TITAN', 'ULTRACEMCO', 'WIPRO', 'ZEEL'],
      dtype=object)

In [5]:
mypf['InPortfolio'].value_counts()

0    46
1    15
Name: InPortfolio, dtype: int64

In [6]:
mypf[(mypf['Symbol'] + '.NS') == (stock_ids[1] + '.NS')]['Forecast'].values[0]

1044.0

In [7]:
# import necessary libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
import talib 
yfin.pdr_override()

In [8]:
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
    #end_date = '2020-01-01'
    end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
    short_window = 20 
    mid_window = 50
    long_window = 200
    moving_avg = 'EMA'


    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = web.DataReader(stock_symbol,  data_source='yahoo', start = start, end = end)['Close']
    stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close Price'].ewm(span = short_window, adjust = False).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close Price'].ewm(span = mid_window, adjust = False).mean(),0)     

    stock_df['RSI'] = talib.RSI(stock_df['Close Price'], timeperiod=14)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close Price'].ewm(span = long_window, adjust = False).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_200'] = round((stock_df['Close Price'] - stock_df['200_EMA'])*100/stock_df['200_EMA'],2)
    stock_df['Max'] = round(max(stock_df['Close Price']),0)
    stock_df['Min'] = round(min(stock_df['Close Price']),0)    
    #stock_df['Dev%_Max'] = round((stock_df['Close Price'] - stock_df['Max'])*100/stock_df['Max'],2)
    #stock_df['Dev%_Min'] = round((stock_df['Close Price'] - stock_df['Min'])*100/stock_df['Min'],2)
    stock_df['Premium%'] = round((stock_df['Close Price']-stock_df['Min'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close Price'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Close Price'] = round(stock_df['Close Price'],0) 
    
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    stock_id = stock_id.upper() + '.NS'
    print(stock_id)
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Forecast'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Forecast'].values[0]
    df_prec_dev = df_prec_dev.append(tmp)

ASIANPAINT.NS
[*********************100%***********************]  1 of 1 completed
AXISBANK.NS
[*********************100%***********************]  1 of 1 completed
BAJFINANCE.NS
[*********************100%***********************]  1 of 1 completed
BERGEPAINT.NS
[*********************100%***********************]  1 of 1 completed
BHARTIARTL.NS
[*********************100%***********************]  1 of 1 completed
BRITANNIA.NS
[*********************100%***********************]  1 of 1 completed
CANBK.NS
[*********************100%***********************]  1 of 1 completed
CIPLA.NS
[*********************100%***********************]  1 of 1 completed
COLPAL.NS
[*********************100%***********************]  1 of 1 completed
DABUR.NS
[*********************100%***********************]  1 of 1 completed
DEEPAKNTR.NS
[*********************100%***********************]  1 of 1 completed
DIVISLAB.NS
[*********************100%***********************]  1 of 1 completed
DMART.NS
[*******************

In [9]:
stock_prec_dev('^NSEI').tail(15)

[*********************100%***********************]  1 of 1 completed


,Close Price,20_EMA,50_EMA,RSI,200_EMA,Stock,Dev%_200,Max,Min,Premium%,Discount%
Date,,,,,,,,,,,
2022-11-04,18117.0,17713.0,17487.0,66.649502,17156.0,^NSEI,5.60,18513.0,15294.0,87.70,12.30
2022-11-07,18203.0,17759.0,17515.0,68.537699,17167.0,^NSEI,6.03,18513.0,15294.0,90.36,9.64
2022-11-09,18157.0,17797.0,17540.0,66.373642,17176.0,^NSEI,5.71,18513.0,15294.0,88.94,11.06
2022-11-10,18028.0,17819.0,17559.0,60.580643,17185.0,^NSEI,4.91,18513.0,15294.0,84.94,15.06
2022-11-11,18350.0,17870.0,17590.0,68.071527,17196.0,^NSEI,6.71,18513.0,15294.0,94.93,5.07
2022-11-14,18329.0,17913.0,17619.0,67.192634,17208.0,^NSEI,6.52,18513.0,15294.0,94.29,5.71
2022-11-15,18403.0,17960.0,17650.0,68.762081,17220.0,^NSEI,6.87,18513.0,15294.0,96.60,3.40
2022-11-16,18410.0,18003.0,17680.0,68.896961,17231.0,^NSEI,6.84,18513.0,15294.0,96.79,3.21
2022-11-17,18344.0,18035.0,17706.0,65.683867,17243.0,^NSEI,6.38,18513.0,15294.0,94.75,5.25


In [10]:
df_prec_dev['Upside%'] = round((df_prec_dev['Forecast']-df_prec_dev['Close Price'])*100/(df_prec_dev['Close Price']),2)
df_prec_dev = df_prec_dev.sort_values(by = 'Upside%', ascending=False)
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)

In [11]:
# buy or accumulate (based on std dev from 200 EMA)
df_prec_dev.sort_values(by = 'Dev%_200', ascending=True).head(20)

,Date,Close Price,20_EMA,50_EMA,RSI,200_EMA,Stock,Dev%_200,Max,Min,Premium%,Discount%,Forecast,Upside%
2,2022-11-25,654.0,680.0,705.0,26.990181,782.0,MOTILALOFS.NS,-16.30,980.0,654.0,0.15,99.85,930.0,42.20
1,2022-11-25,452.0,483.0,506.0,23.500681,532.0,ICICIPRULI.NS,-15.02,616.0,447.0,3.02,96.98,647.0,43.14
5,2022-11-25,3343.0,3393.0,3509.0,41.868500,3847.0,DIVISLAB.NS,-13.11,4938.0,3266.0,4.58,95.42,4065.0,21.60
35,2022-11-25,403.0,394.0,397.0,60.364470,459.0,WIPRO.NS,-12.24,722.0,375.0,8.01,91.99,433.0,7.44
14,2022-11-25,82.0,80.0,79.0,53.909026,92.0,IBREALEST.NS,-11.20,179.0,60.0,18.24,81.76,95.0,15.85
6,2022-11-25,75.0,74.0,74.0,55.477811,84.0,NATIONALUM.NS,-10.24,130.0,67.0,13.33,86.67,91.0,21.33
0,2022-11-25,430.0,418.0,429.0,58.437159,472.0,GODREJIND.NS,-8.98,644.0,405.0,10.29,89.71,638.0,48.37
4,2022-11-25,3955.0,3940.0,3967.0,50.528443,4280.0,NAUKRI.NS,-7.60,5953.0,3375.0,22.48,77.52,4851.0,22.65
36,2022-11-25,1080.0,1056.0,1053.0,58.526281,1168.0,TECHM.NS,-7.52,1806.0,965.0,13.69,86.31,1156.0,7.04
37,2022-11-25,3904.0,4055.0,4160.0,29.889654,4196.0,DMART.NS,-6.95,4894.0,3231.0,40.50,59.50,4177.0,6.99


In [12]:
# sell or downsize (based on std dev from 200 EMA)
df_prec_dev.sort_values(by = 'Dev%_200', ascending=False).head(15)

,Date,Close Price,20_EMA,50_EMA,RSI,200_EMA,Stock,Dev%_200,Max,Min,Premium%,Discount%,Forecast,Upside%
56,2022-11-25,327.0,304.0,278.0,78.503154,240.0,CANBK.NS,36.19,329.0,175.0,98.60,1.40,310.0,-5.20
21,2022-11-25,133.0,133.0,128.0,50.896013,111.0,FEDERALBNK.NS,19.41,138.0,81.0,90.44,9.56,149.0,12.03
8,2022-11-25,888.0,863.0,832.0,64.896469,759.0,AXISBANK.NS,16.96,915.0,626.0,90.57,9.43,1044.0,17.57
11,2022-11-25,607.0,594.0,572.0,63.307056,526.0,SBIN.NS,15.48,616.0,435.0,95.25,4.75,709.0,16.80
9,2022-11-25,1187.0,1158.0,1145.0,59.431047,1041.0,INDUSINDBK.NS,14.03,1264.0,778.0,84.18,15.82,1394.0,17.44
49,2022-11-25,1034.0,1013.0,981.0,64.082112,908.0,SUNPHARMA.NS,13.86,1052.0,737.0,94.24,5.76,1056.0,2.13
45,2022-11-25,340.0,344.0,338.0,45.167895,299.0,ITC.NS,13.81,361.0,208.0,86.47,13.53,350.0,2.94
57,2022-11-25,4184.0,4053.0,3917.0,69.878735,3690.0,BRITANNIA.NS,13.38,4191.0,3094.0,99.33,0.67,3935.0,-5.95
42,2022-11-25,847.0,831.0,804.0,63.137041,750.0,BHARTIARTL.NS,12.91,851.0,641.0,98.02,1.98,879.0,3.78
15,2022-11-25,930.0,916.0,897.0,60.860026,824.0,ICICIBANK.NS,12.90,939.0,654.0,96.95,3.05,1075.0,15.59


In [13]:
# portfolio buy/sell
mypf['Stock'] = mypf['Symbol'] + '.NS'
df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]

,Date,Close Price,20_EMA,50_EMA,RSI,200_EMA,Stock,Dev%_200,Max,Min,Premium%,Discount%,Forecast,Upside%
0,2022-11-25,430.0,418.0,429.0,58.437159,472.0,GODREJIND.NS,-8.98,644.0,405.0,10.29,89.71,638.0,48.37
1,2022-11-25,452.0,483.0,506.0,23.500681,532.0,ICICIPRULI.NS,-15.02,616.0,447.0,3.02,96.98,647.0,43.14
2,2022-11-25,654.0,680.0,705.0,26.990181,782.0,MOTILALOFS.NS,-16.30,980.0,654.0,0.15,99.85,930.0,42.20
3,2022-11-25,1235.0,1245.0,1247.0,45.457528,1204.0,SBILIFE.NS,2.60,1332.0,1025.0,68.49,31.51,1577.0,27.69
4,2022-11-25,3955.0,3940.0,3967.0,50.528443,4280.0,NAUKRI.NS,-7.60,5953.0,3375.0,22.48,77.52,4851.0,22.65
5,2022-11-25,3343.0,3393.0,3509.0,41.868500,3847.0,DIVISLAB.NS,-13.11,4938.0,3266.0,4.58,95.42,4065.0,21.60
6,2022-11-25,75.0,74.0,74.0,55.477811,84.0,NATIONALUM.NS,-10.24,130.0,67.0,13.33,86.67,91.0,21.33
13,2022-11-25,587.0,544.0,543.0,74.287945,567.0,HDFCLIFE.NS,3.46,705.0,504.0,41.09,58.91,683.0,16.35
14,2022-11-25,82.0,80.0,79.0,53.909026,92.0,IBREALEST.NS,-11.20,179.0,60.0,18.24,81.76,95.0,15.85
16,2022-11-25,1907.0,1812.0,1817.0,66.229803,1861.0,INDIGO.NS,2.47,2265.0,1549.0,49.99,50.01,2186.0,14.63


In [14]:
df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]['Upside%']*100/df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]['Upside%'].sum()

0     16.335146
1     14.568910
2     14.251461
3      9.351255
4      7.649184
5      7.294586
6      7.203404
13     5.521597
14     5.352741
16     4.940731
22     4.042417
33     2.664550
44     1.205633
51    -0.050657
53    -0.330958
Name: Upside%, dtype: float64

In [15]:
# prospects buy
df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]

,Date,Close Price,20_EMA,50_EMA,RSI,200_EMA,Stock,Dev%_200,Max,Min,Premium%,Discount%,Forecast,Upside%
7,2022-11-25,6748.0,6937.0,7043.0,36.252230,6883.0,BAJFINANCE.NS,-1.97,7853.0,5284.0,56.97,43.03,7978.0,18.23
8,2022-11-25,888.0,863.0,832.0,64.896469,759.0,AXISBANK.NS,16.96,915.0,626.0,90.57,9.43,1044.0,17.57
9,2022-11-25,1187.0,1158.0,1145.0,59.431047,1041.0,INDUSINDBK.NS,14.03,1264.0,778.0,84.18,15.82,1394.0,17.44
10,2022-11-25,556.0,566.0,582.0,41.831015,597.0,JUBLFOOD.NS,-6.83,786.0,464.0,28.65,71.35,651.0,17.09
11,2022-11-25,607.0,594.0,572.0,63.307056,526.0,SBIN.NS,15.48,616.0,435.0,95.25,4.75,709.0,16.80
12,2022-11-25,259.0,258.0,260.0,50.035303,265.0,ZEEL.NS,-2.11,371.0,209.0,31.11,68.89,302.0,16.60
15,2022-11-25,930.0,916.0,897.0,60.860026,824.0,ICICIBANK.NS,12.90,939.0,654.0,96.95,3.05,1075.0,15.59
17,2022-11-25,433.0,423.0,423.0,58.046049,439.0,TATAMOTORS.NS,-1.32,525.0,372.0,40.00,60.00,493.0,13.86
18,2022-11-25,2109.0,2166.0,2157.0,41.087317,2101.0,DEEPAKNTR.NS,0.36,2660.0,1720.0,41.34,58.66,2371.0,12.42
19,2022-11-25,557.0,551.0,552.0,55.963364,554.0,DABUR.NS,0.46,606.0,485.0,59.13,40.87,625.0,12.21


In [16]:
# CCI buy
df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['CCIFlag'] == 1].Stock.values)]

,Date,Close Price,20_EMA,50_EMA,RSI,200_EMA,Stock,Dev%_200,Max,Min,Premium%,Discount%,Forecast,Upside%
5,2022-11-25,3343.0,3393.0,3509.0,41.868500,3847.0,DIVISLAB.NS,-13.11,4938.0,3266.0,4.58,95.42,4065.0,21.60
10,2022-11-25,556.0,566.0,582.0,41.831015,597.0,JUBLFOOD.NS,-6.83,786.0,464.0,28.65,71.35,651.0,17.09
24,2022-11-25,2537.0,2520.0,2543.0,51.654315,2451.0,HINDUNILVR.NS,3.53,2702.0,1944.0,78.29,21.71,2828.0,11.47
31,2022-11-25,3108.0,3113.0,3172.0,46.251015,3161.0,ASIANPAINT.NS,-1.67,3576.0,2580.0,53.03,46.97,3361.0,8.14
33,2022-11-25,1636.0,1567.0,1527.0,68.277423,1562.0,INFY.NS,4.72,1940.0,1365.0,47.07,52.93,1765.0,7.89
35,2022-11-25,403.0,394.0,397.0,60.364470,459.0,WIPRO.NS,-12.24,722.0,375.0,8.01,91.99,433.0,7.44
36,2022-11-25,1080.0,1056.0,1053.0,58.526281,1168.0,TECHM.NS,-7.52,1806.0,965.0,13.69,86.31,1156.0,7.04
39,2022-11-25,1246.0,1235.0,1251.0,53.167414,1262.0,HAVELLS.NS,-1.23,1418.0,1070.0,50.72,49.28,1327.0,6.50
40,2022-11-25,617.0,607.0,613.0,55.974657,647.0,BERGEPAINT.NS,-4.68,791.0,561.0,24.22,75.78,651.0,5.51
41,2022-11-25,2595.0,2635.0,2620.0,42.872709,2476.0,TITAN.NS,4.82,2770.0,1936.0,79.06,20.94,2715.0,4.62
